# Fine-tuning to Memorize Knowledge

In this tutorial we experiment with some basic approaches of "baking in knowledge with fine-tuning."

- Synthesizing questions from existing context
- Trying text completion

In [1]:
import os
import openai
from llama_index import ServiceContext
from llama_index.llms import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

## Load Data

In [3]:
!mkdir data && wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

mkdir: data: File exists


In [2]:
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader
from llama_hub.file.unstructured.base import UnstructuredReader
from llama_hub.file.pymu_pdf.base import PyMuPDFReader

In [3]:
loader = PyMuPDFReader()
docs0 = loader.load(file_path=Path("./data/llama2.pdf"))

In [4]:
from llama_index import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
metadata = {
    "paper_title": "Llama 2: Open Foundation and Fine-Tuned Chat Models"
}
docs = [Document(text=doc_text, metadata=metadata)]

In [ ]:
print(docs[0].get_content())

In [20]:
from llama_index.callbacks import CallbackManager
callback_manager = CallbackManager([])

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0.3),
    callback_manager=callback_manager
)
gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-0613", temperature=0.3),
    callback_manager=callback_manager
)

## Generate Dataset

In [21]:
from llama_index.evaluation import DatasetGenerator
from llama_index.node_parser import SimpleNodeParser
# try evaluation modules
from llama_index.evaluation import QueryResponseEvaluator, ResponseEvaluator
from llama_index import PromptTemplate

In [22]:
node_parser = SimpleNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(docs)

In [25]:
from tqdm.notebook import tqdm
import json

num_questions_per_chunk = 10
question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context, "
    f"formulate {num_questions_per_chunk} that captures an important fact from the "
    "context. \n"
    "You MUST obey the following criteria:\n"
    "- Restrict the question to the context information provided.\n"
    "- Do NOT create a question that cannot be answered from the context.\n"
    "- Phrase the question so that it does NOT refer to specific context. "
    "For instance, do NOT put phrases like \"given provided context\" or \"in this work\" in the question, "
    "because if the question is asked elsewhere it wouldn't be provided specific context. Replace these terms "
    "with specific details.\n"
    "BAD questions:\n"
    "What did the author do in his childhood\n"
    "What were the main findings in this report\n\n"
    "GOOD questions:\n"
    "What did Barack Obama do in his childhood\n"
    "What were the main findings in the original Transformers paper by Vaswani et al.\n\n"
    "Generate the questions below:\n"
)

# go through each node one at a time - 
# generate questions, filter using eval modules, and dump to file 

fp = open("data/qa_pairs.jsonl", "w")
for idx, node in enumerate(nodes):
    dataset_generator = DatasetGenerator(
        [node],
        question_gen_query=question_gen_query,
        service_context=gpt_4_context,
        metadata_mode="all"
    )
    node_questions_0 = dataset_generator.generate_questions_from_nodes(num=10)
    print(f"[Node {idx}] Generated questions:\n {node_questions_0}")
    # for each question, get a response
    for question in tqdm(node_questions_0):
        index = SummaryIndex([node], service_context=gpt_35_context)  
        query_engine = index.as_query_engine()
        response = query_engine.query(question)
        out_dict = {
            "query": question,
            "response": str(response)
        }
        print(f"[Node {idx}] Outputs: {out_dict}")
        fp.write(json.dumps(out_dict) + "\n")

fp.close()
    

[Node 0] Generated questions:
 ['What is the name of the collection of pretrained and fine-tuned large language models discussed in the paper?', 'What is the range of parameters in the Llama 2 models?', 'What is the specific use case for the Llama 2-Chat models?', 'How do the Llama 2 models perform compared to open-source chat models on most benchmarks?', 'What are the two main areas of focus in the development of Llama 2-Chat models?', 'What is the purpose of providing a detailed description of the fine-tuning and safety improvements of Llama 2-Chat?', 'What is the scale of the large language models developed in Llama 2?', 'What is the potential substitute for closed-source models according to the paper?', 'What are the two methods used for fine-tuning in Llama 2?', 'What is the significance of safety in the development of Llama 2 models?']


  0%|          | 0/10 [00:00<?, ?it/s]

[Node 0] Outputs: {'query': 'What is the name of the collection of pretrained and fine-tuned large language models discussed in the paper?', 'response': 'The name of the collection of pretrained and fine-tuned large language models discussed in the paper is Llama 2.'}
[Node 0] Outputs: {'query': 'What is the range of parameters in the Llama 2 models?', 'response': 'The range of parameters in the Llama 2 models is from 7 billion to 70 billion.'}
[Node 0] Outputs: {'query': 'What is the specific use case for the Llama 2-Chat models?', 'response': 'The Llama 2-Chat models are optimized for dialogue use cases.'}
[Node 0] Outputs: {'query': 'How do the Llama 2 models perform compared to open-source chat models on most benchmarks?', 'response': 'The Llama 2 models outperform open-source chat models on most benchmarks.'}
[Node 0] Outputs: {'query': 'What are the two main areas of focus in the development of Llama 2-Chat models?', 'response': 'The two main areas of focus in the development of 

  0%|          | 0/10 [00:00<?, ?it/s]

[Node 1] Outputs: {'query': 'What is the title of the section that discusses the safety evaluation of the Llama 2-Chat?', 'response': 'The title of the section that discusses the safety evaluation of the Llama 2-Chat is "Safety Evaluation of Llama 2-Chat".'}
[Node 1] Outputs: {'query': 'What is the content of section 4.3 in the document?', 'response': 'The content of section 4.3 in the document is "Red Teaming".'}
[Node 1] Outputs: {'query': 'What topic is covered in section 5.2 of the document?', 'response': 'The topic covered in section 5.2 of the document is "Limitations and Ethical Considerations."'}
[Node 1] Outputs: {'query': 'What is the focus of the discussion in section 5.3?', 'response': 'The focus of the discussion in section 5.3 is the responsible release strategy.'}
[Node 1] Outputs: {'query': 'What is the subject of section 6 in the document?', 'response': 'The subject of section 6 in the document is "Related Work."'}
[Node 1] Outputs: {'query': 'What is the final section

...

[Node 84] Outputs: {'query': 'What is considered as "contaminated" in an evaluation set according to the 2022 study?', 'response': 'According to the 2022 study, an evaluation set is considered "contaminated" if there exists a collision between a high-order n-gram (generally, n = 13) from the sample and the training data. This approach is used to produce a "clean" subset of the data with high precision.'}
[Node 84] Outputs: {'query': 'What approach is used to produce a "clean" subset of the data with high precision?', 'response': 'The approach used to produce a "clean" subset of the data with high precision is by considering a collision between a high-order n-gram (generally, n = 13) from the sample and the training data. This approach is deliberately conservative and is used in open-sourced evaluation libraries.'}
[Node 84] Outputs: {'query': 'What is the limitation of the approach used to measure dataset contamination?', 'response': 'The limitation of the approach used to measure data

  0%|          | 0/10 [00:00<?, ?it/s]

[Node 85] Outputs: {'query': 'What does the term "contamination" refer to in the context of training data?', 'response': 'The term "contamination" in the context of training data refers to the presence of matched sequences in the training data that may appear to be contaminated. However, it is unlikely that the model actually saw the correctly-assembled contaminated sequences during training. The analysis is done to determine if the training data has been affected by contamination and if it has impacted the evaluation performance of the model.'}
[Node 85] Outputs: {'query': 'What is the significance of the statistic Zn in the contamination analysis?', 'response': 'The significance of the statistic Zn in the contamination analysis is that it measures the deviation of the mean performance metric (¯X) from the expected mean (µn) in terms of the standard deviation (σn). By calculating Zn, the analysis determines whether there is sufficient evidence to suggest that contamination has affecte

  0%|          | 0/10 [00:00<?, ?it/s]

[Node 86] Outputs: {'query': 'Who are the developers of the Llama 2 model?', 'response': 'The developers of the Llama 2 model are Meta AI.'}
[Node 86] Outputs: {'query': 'What are the different parameter sizes that Llama 2 comes in?', 'response': 'Llama 2 comes in a range of parameter sizes, including 7B, 13B, and 70B.'}
[Node 86] Outputs: {'query': 'What type of architecture does the Llama 2 model use?', 'response': 'The Llama 2 model uses an optimized transformer architecture.'}
[Node 86] Outputs: {'query': 'During which months was the Llama 2 model trained?', 'response': 'The Llama 2 model was trained between January 2023 and July 2023.'}
[Node 86] Outputs: {'query': 'What is the intended use of the Llama 2 model?', 'response': 'The intended use of the Llama 2 model is for commercial and research purposes in English. The pretrained models can be adapted for various natural language generation tasks, while the tuned models are specifically intended for assistant-like chat.'}
[Node 86

### Filter out questions using QueryResponseEvaluator

Do a second pass to make sure only questions that can be answerd by context make it into the training set.

In [27]:
# try evaluation modules
from llama_index.evaluation import QueryResponseEvaluator, ResponseEvaluator
from llama_index import PromptTemplate
from llama_index.llms import OpenAI

In [36]:
query_eval_tmpl = PromptTemplate(
    "Your task is to evaluate the following: If the response for the query isn't able to answer the question provided.\n"
    "If query isn't able to answer the question, answer NO.\n"
    "Otherwise answer YES.\n"
    "To elaborate, you might get an answer like the following: 'The context does not contain the answer to this question.'"
    "Please return NO in that case. "
    "You be given the query and response. Return YES or NO as the answer.\n"
    "Query: \n {query_str}\n"
    "Response: \n {response_str}\n"
    "Answer: "
)

eval_llm = OpenAI(model="gpt-4-0613")

In [45]:
def filter_data(path: str, out_path: str):
    fp = open(path, "r")
    out_fp = open(out_path, "w")
    new_lines = []
    for idx, line in enumerate(fp):
        qa_pair = json.loads(line)
        eval = eval_llm.complete(query_eval_tmpl.format(query_str=qa_pair["query"], response_str=qa_pair["response"]))
        
        print(f"[{idx}] QA Pair: {qa_pair} \n Eval: {eval}")
        if "NO" in eval:
            continue
        else:
            # new_lines.append(line)
            out_fp.write(line)

In [ ]:
filter_data("data/qa_pairs.jsonl", "data/qa_pairs_2.jsonl")

### Split into Training and Validation Sets

We split into training and validation sets.

**NOTE**: We shuffle the data before splitting. This helps ensure that the training data has coverage throughout the document.

In [130]:
from copy import deepcopy
import random

def split_train_val(path: str, out_train_path: str, out_val_path: str, train_split=0.7):
    with open(path, "r") as fp:
        lines = fp.readlines()

        # shuffle the lines to make sure that the "train questions" cover most fo the context
        shuffled_lines = deepcopy(lines)
        random.shuffle(shuffled_lines)
        
        split_idx = int(train_split * len(shuffled_lines))
        train_lines = shuffled_lines[:split_idx]
        val_lines = shuffled_lines[split_idx:]
        with open(out_train_path, "w") as out_fp:
            out_fp.write("".join(train_lines))

        with open(out_val_path, "w") as out_fp:
            out_fp.write("".join(val_lines))

In [132]:
split_train_val("data/qa_pairs_2.jsonl", "data/qa_pairs_train.jsonl", "data/qa_pairs_val.jsonl")

### Format into Training Data

Format into training data for OpenAI's finetuning endpoints.

**NOTE**: We don't use our `OpenAIFinetuningHandler` because that logs the full input prompt including context as the user message. Here we just want to log the query as the user message, because we want to fine-tune gpt-3.5-turbo to "bake in knowledge" into the fine-tuned model.

In [133]:
fp = open("data/qa_pairs_train.jsonl", "r")
out_fp = open("data/qa_pairs_openai.jsonl", "w")
# TODO: try with different system prompts
system_prompt = {"role": "system", "content": "You are a helpful assistant helping to answer questions about the Llama 2 paper."}
for line in fp:
    qa_pair = json.loads(line)
    user_prompt = {"role": "user", "content": qa_pair["query"]}
    assistant_prompt = {"role": "assistant", "content": qa_pair["response"]}
    out_dict = {
        "messages": [system_prompt, user_prompt, assistant_prompt],
    }
    out_fp.write(json.dumps(out_dict) + "\n")

## Fine-tune the Model

In [214]:
from llama_index.finetuning import OpenAIFinetuneEngine

In [215]:
finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "data/qa_pairs_openai.jsonl",
    # start_job_id="<start-job-id>"  # if you have an existing job, can specify id here
)

In [216]:
finetune_engine.finetune()

Num examples: 597
First example:
{'role': 'system', 'content': 'You are a helpful assistant helping to answer questions about the Llama 2 paper.'}
{'role': 'user', 'content': 'Who were the early reviewers of the paper on "Llama 2: Open Foundation and Fine-Tuned Chat Models" who helped improve its quality?'}
{'role': 'assistant', 'content': 'Mike Lewis, Joelle Pineau, Laurens van der Maaten, Jason Weston, and Omer Levy were the early reviewers of the paper on "Llama 2: Open Foundation and Fine-Tuned Chat Models" who helped improve its quality.'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 50, 637
mean / median: 102.51256281407035, 90.0
p5 / p95: 66.0, 155.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 588
mean / median: 50.45728643216081, 35.0
p5 /

In [225]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-fk0428lntJCRh6x1GKeccv8E at 0x2b95fd6c0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-fk0428lntJCRh6x1GKeccv8E",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1694406904,
  "finished_at": 1694409009,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:llamaindex::7xTTW0hT",
  "organization_id": "org-1ZDAvajC6v2ZtAP9hLEIsXRz",
  "result_files": [
    "file-Ao1r7cGnYJbHqCG79zAQo6lP"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-9ndBjJX0pZ3Do4mPhADcTOef",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 180006,
  "error": null
}

In [226]:
ft_model = finetune_engine.get_finetuned_model()

In [227]:
ft_model

OpenAI(callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x2bdaba2f0>, model='ft:gpt-3.5-turbo-0613:llamaindex::7xTTW0hT', temperature=0.1, max_tokens=None, additional_kwargs={}, max_retries=10, class_type='openai')

## Evaluate Results

We run evaluations, over both the validation set but also the training set.

**Wait, isn't evaluating over the training set cheating?**

- It's a sanity check of how much the model was able to memorize information it's trained on.
- The training data contains quite a bit of content about the paper, so by answering the training set well the model would at least be well-equipped to answer some questions.

In [258]:
from llama_index.llms import ChatMessage

In [234]:
def load_data(path: str):
    fp = open(path, "r")
    data_dicts = []
    for line in fp:
        d = json.loads(line)
        data_dicts.append(d)
    return data_dicts


In [260]:
train_dicts = load_data("data/qa_pairs_train.jsonl")
eval_dicts = load_data("data/qa_pairs_val.jsonl")

In [269]:
def query_model(model, d):
    # print(d)
    msgs = [
        ChatMessage(role="system", content="You are a helpful assistant helping to answer questions about the Llama 2 paper."),
        ChatMessage(role="user", content=d["query"]),
    ]

    # try ft-model
    response = model.chat(msgs)
    return str(response)

In [254]:
response = query_model(ft_model, eval_dicts[7])
print(eval_dicts[7])
print(response)

{'query': 'What is the title of the paper discussed in the context?', 'response': 'The title of the paper discussed in the context is "Llama 2: Open Foundation and Fine-Tuned Chat Models".'}


'assistant: The title of the paper discussed in the context is "Llama 2: Open Foundation and Fine-Tuned Chat Models".'

In [261]:
query_model(ft_model, train_dicts[7])
print(train_dicts[7])
print(response)

{'query': 'How is the decision made whether to use safety context distillation or not?', 'response': 'The decision to use safety context distillation is made based on the reward model score. The safety reward model is leveraged to determine whether the context-distilled output receives a better reward model score than the original answer. If the context-distilled output gets a better reward model score, it is kept. This approach helps limit the negative impact of context distillation while still utilizing it in cases where it improves the reward model score.'}


'assistant: The decision to use safety context distillation is made based on the reward model score. If the reward model score is below a certain threshold, safety context distillation is used.'

### Setup Baseline RAG system to benchmark

We setup a baseline RAG system powered by gpt-3.5-turbo to help benchmark the quality of results.

In [262]:
# baseline RAG system
from llama_index import VectorStoreIndex
base_index = VectorStoreIndex(nodes, service_context=gpt_35_context)
base_query_engine = base_index.as_query_engine()

In [252]:
# baseline model
base_model = OpenAI(model="gpt-4", temperature=0.3)

In [257]:
query_model(base_model, eval_dicts[80])

{'query': 'How does Llama 2-Chat address the issue of spreading misinformation or conspiracy theories?', 'response': "Llama 2-Chat addresses the issue of spreading misinformation or conspiracy theories by refuting any misinformation in the prompt immediately. It emphasizes the importance of relying on scientific evidence and credible sources when evaluating historical events. The model does not promote or encourage the spread of false information and instead focuses on sharing accurate and helpful information. It also highlights the importance of fact-checking and critical thinking when assessing the validity of a claim. Llama 2-Chat's programming rules prioritize respect for truth and accuracy in all forms of communication and discourage the spread of misinformation or conspiracy theories."}


"assistant: The Llama 2 paper does not specifically address the issue of spreading misinformation or conspiracy theories. However, it does mention that the model is designed to refuse outputs that are inappropriate or harmful. This could potentially include misinformation or conspiracy theories. It also states that the model's responses are based on a mixture of licensed data, data created by human trainers, and publicly available data. The developers have also used reinforcement learning from human feedback to fine-tune the model, which can help in reducing the spread of false information. However, the specifics of how misinformation or conspiracy theories are handled are not detailed in the paper."

### Run Evaluations

We log the responses from the fine-tuned model, the baseline RAG system, and the baseline model.

We then run all responses through a GPT-4 prompt, comparing each against the ground-truth to measure validity of the result.

In [278]:
import pandas as pd
from tqdm.notebook import tqdm

EVAL_PROMPT_TMPL = PromptTemplate("""\
We provide a question and the 'ground-truth' answer. We also provide \
the predicted answer.

Evaluate whether the predicted answer is correct, given its similarity \
to the ground-truth. If details provided in predicted answer are reflected \
in the ground-truth answer, return "YES". To return "YES", the details don't \
need to exactly match. Be lenient in evaluation if the predicted answer \
is missing a few details. Try to make sure that there are no blatant mistakes. \
Otherwise, return "NO".

Question: {question}
Ground-truth Answer: {gt_answer}
Predicted Answer: {pred_answer}
Evaluation Result: \
""")


def eval_match_gt(query, gt_response, pred_response):
    llm = OpenAI(model="gpt-4", temperature=0.0)
    fmt_prompt = EVAL_PROMPT_TMPL.format(
        question=query,
        gt_answer=gt_response,
        pred_answer=pred_response,
    )
    result = llm.complete(fmt_prompt)
    if "yes" in str(result).lower():
        return 1
    else:
        return 0


def run_evals(eval_dicts):
    """Run evals - fine-tuned model, RAG system, and base model."""

    raw_responses = []
    for eval_dict in tqdm(eval_dicts):
        gt_response = eval_dict["response"]
        ft_response = str(query_model(ft_model, eval_dict))
        rag_response = str(base_query_engine.query(eval_dict["query"]))
        base_response = str(query_model(base_model, eval_dict))

 
        # try evaluations
        ft_eval = eval_match_gt(eval_dict["query"], gt_response, ft_response)
        rag_eval = eval_match_gt(eval_dict["query"], gt_response, rag_response)
        base_eval = eval_match_gt(eval_dict["query"], gt_response, base_response)

        response_dict = {
            "query": eval_dict["query"],
            "gt_response": gt_response,
            "ft_response": ft_response,
            "rag_response": rag_response,
            "base_response": base_response,
            "ft_eval": ft_eval,
            "rag_eval": rag_eval,
            "base_eval": base_eval
        }

        raw_responses.append(response_dict)

    raw_responses_df = pd.DataFrame(raw_responses)

    eval_dict = {
        "ft_score": raw_responses_df["ft_eval"].mean(),
        "rag_score": raw_responses_df["rag_eval"].mean(),
        "base_score": raw_responses_df["base_eval"].mean()
    }
    
    return eval_dict, raw_responses_df

In [274]:
pd.set_option('display.max_colwidth', None)

#### Qualitative Evaluations

Here we show some qualitative output examples over both the training and validation sets.

In [275]:
eval_dict, raw_response_df = run_evals(train_dicts[7:8])
display(eval_dict)
display(raw_response_df)

{'ft_score': 1.0, 'rag_score': 1.0, 'base_score': 0.0}

,query,gt_response,ft_response,rag_response,base_response,ft_eval,rag_eval,base_eval
0,How is the decision made whether to use safety context distillation or not?,"The decision to use safety context distillation is made based on the reward model score. The safety reward model is leveraged to determine whether the context-distilled output receives a better reward model score than the original answer. If the context-distilled output gets a better reward model score, it is kept. This approach helps limit the negative impact of context distillation while still utilizing it in cases where it improves the reward model score.","assistant: The decision to use safety context distillation is made based on the reward model score. If the reward model score is above a certain threshold, safety context distillation is used.","The decision to use safety context distillation is made based on the reward model score. The safety reward model is used to evaluate whether the context-distilled output gets a better reward model score than the original answer. If the context-distilled output receives a better reward model score, it is kept. This approach helps limit the negative impact of context distillation while still improving the safety of the model's responses.","assistant: The Llama 2 paper does not provide specific criteria for deciding when to use safety context distillation. The choice to use this method would likely depend on the specific requirements of the task and the potential risks involved. Safety context distillation is used to ensure that the model behaves safely even in situations that were not covered in the training data. If the task involves high-risk decisions or is in a domain where unexpected situations are likely to occur, it might be more important to use safety context distillation. However, this would likely be a decision made on a case-by-case basis, considering factors such as the complexity of the task, the quality and diversity of the training data, and the potential consequences of unsafe behavior.",1,1,0


In [276]:
eval_dict, raw_response_df = run_evals(eval_dicts[6:7])
display(eval_dict)
display(raw_response_df)

{'ft_score': 0.0, 'rag_score': 1.0, 'base_score': 0.0}

,query,gt_response,ft_response,rag_response,base_response,ft_eval,rag_eval,base_eval
0,What model is used to predict the truthfulness and informativeness of the generated outputs from LLMs?,"A fine-tuned GPT-3 model, referred to as ""GPT-judge,"" is used to predict the truthfulness and informativeness of the generated outputs from LLMs.",assistant: The model used to predict the truthfulness and informativeness of the generated outputs from LLMs is called TruthfulQA.,"A fine-tuned GPT-3 model, referred to as ""GPT-judge,"" is used to predict the truthfulness and informativeness of the generated outputs from LLMs.","assistant: The Llama 2 paper does not specify a particular model used to predict the truthfulness and informativeness of the generated outputs from LLMs (Language Model). The paper primarily focuses on the limitations and potential risks of large language models. If you're referring to a different paper or model, please provide more details.",0,1,0


#### Quantitative Evaluations

Here we show quantitative metrics over both the training and eval set.

In [281]:
import random
k = 40

train_dicts_sample = random.sample(train_dicts, k)
eval_dicts_sample = random.sample(eval_dicts, k)

In [282]:
eval_result, raw_response_df = run_evals(train_dicts_sample)
display(eval_result)
# display(raw_response_df)

  0%|          | 0/40 [00:00<?, ?it/s]

{'ft_score': 0.425, 'rag_score': 0.7, 'base_score': 0.225}

In [ ]:
eval_result, raw_response_df = run_evals(eval_dicts_sapmle)
display(eval_result)
# display(raw_response_df)